In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import welch

from speckit import compute_spectrum
from speckit.noise import band_limited_noise
from multiprocessing import Pool

In [ ]:
# Generate sample data:
N  = int(1e6)           # Length of the example time series
fs = 2.                 # Sampling frequency
x = np.random.rand(N)  # Generate random data of the given length

# Calculate spectrum via Welch's method:
f1, psd = welch(x, fs, window=("kaiser", 30), nperseg=N//4)

# Calculate spectrum via SpecKit:
obj = compute_spectrum(
    x,
    fs,
    order=0,         # -1: pass through, 0: remove mean, 1...n: polynomial detrend
    Jdes=1000,       # Target number of frequencies
    Kdes=100,        # Control parameter for the scheduler
    bmin=1.0,        # Minimum fractional bin number to consider
    Lmin=20,         # Minimum length of time series segments
    win="Kaiser",    # Window function to use
    olap="default",  # Fractional segment overlap ("default" will use optimal value for the given window)
    psll=200,        # Peak sidelobe level suppression (used to calculate optimal Kaiser window parameters)
    pool=Pool()      # multiprocessing Pool
)

In [ ]:
fig, ax = plt.subplots(figsize=(6,4), dpi=150)
ax.set_xlabel('Fourier frequency (Hz)')
ax.set_ylabel(r'Spectral density (units $/\sqrt{\rm Hz}$)')
ax.loglog(f1, np.sqrt(psd), label="Welch", color='gray', alpha=0.5)
obj.plot(which='asd', errors=True, sigma=3, ax=ax, label='SpecKit')
ax.set_xlim(obj.f[0], obj.f[-1])
ax.set_ylim(1e-3,2)
plt.show()